# Adding Dependeces

In [2]:
using DelimitedFiles
using CUDA
using ProgressMeter
include("../../src/forces.jl")
include("../../src/Neighbor.jl")

knn_cu (generic function with 1 method)

# Initial Parameters

# Functions

In [4]:
function init()
    # Definig Variables for calculing initial variables
    global X

    # Inizializate Variables for kNN
    global i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
    global Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
    global idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu

    # Inizializate Variables for Forces
    global r = zeros(nn,size(X)[1],3) |> cu
    global dist = zeros(nn, size(X)[1]) |> cu
    global F = zeros(nn, size(X)[1],3) |> cu
    global dX = zeros(size(X)[1],3) |> cu;
end

function calc(n_knn, t_f, r_max, s, K)
    init()

    p = Progress(Int(t_f/dt),barlen=25)
    for i in 0:Int(t_f/dt)

        if mod(i, n_knn) == 0
            # Calculating kNN
            knn_cu()
        end

        # Calculating Forces
        forces_cu(r_max, s, K)

        global X = X + dX*dt
        next!(p)
    end
end

function main(t_f, r_max, s, K)
    # Initial Coordinates of Aggregates
    global X = Float32.(readdlm("../../../../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

    X = Matrix(X)
    X_f = zeros(size(X,1))
    for i in 1:size(X_f,1)
        A = sqrt(X[i,1]^2+X[i,2]^2+X[i,3]^2)
        if A > 0.8*R_agg
            X_f[i] = 1
        else
            X_f[i] = 2
        end
    end

    X_f_2 = Int.(vcat(X_f,X_f));

    # Fusioning two Spheres
    Size = Float32((findmax(X[:,1])[1] - findmin(X[:,1])[1])/2 + 1)
    X_2 = vcat(X,X) |> cu
    X_2[1:size(X,1),1] = X_2[1:size(X,1),1] .- (Size)
    X_2[size(X,1):end,1] = X_2[size(X,1):end,1] .+ Size

    open("Test_Initial.xyz"; write=true) do f
        write(f, "$(size(X_2, 1))\n")
        write(f, "Initial Data ($(R_agg))\n")
        writedlm(f,hcat(X_f_2, X_2), ' ')
    end

    X = X_2 |> cu

    # Running Forces Function
    calc(n_knn, t_f, r_max, s, K)

    X = Matrix(X)
    open("Test_Final.xyz"; write=true) do f
        write(f, "$(size(X, 1))\n")
        write(f, "Initial Data ($(R_agg))\n")
        writedlm(f,hcat(X_f_2, X), ' ')
    end

    # run(`paraview Test_Initial.xyz Test_Final.xyz`)
end

main (generic function with 1 method)

# Stabilizing One Sphere

In [3]:
# X = X |> cu
# main(n_knn)

# X = Matrix(X)
# @time open("Test_Zero.xyz"; write=true) do f
#     write(f, "$(size(X, 1))\n")
#     write(f, "Initial Data ($(R_agg))\n")
#     writedlm(f,hcat(X_f, X), ' ')
# end
# X = X |> cu

# run(`paraview Test_Zero.xyz`)

# Fusioning tho Spheres

In [7]:
# Physical Conditions
R_agg = 10

# Time model Conditions
dt = 0.1
n_knn = 50
nn = 16

# --------------------------------

T_f = 150000
R_Max = 3.5
S = 1.9
K_array = [0.005]  

for t_f in T_f
    for r_max in R_Max
        for s in S
            for K in K_array
                #Generating the path
                if !isdir("T_$(t_f)") mkdir("T_$(t_f)") end
                if !isdir("T_$(t_f)/rmax_$(r_max)_s_$(s)") mkdir("T_$(t_f)/rmax_$(r_max)_s_$(s)") end
                if !isdir("T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)") mkdir("T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)")  end
                
                
                # Calculating all above
                if size(readdir("T_$(T_f)/rmax_$(R_Max)_s_$(S)/k_$(K)"))[1] == 0
                    cd("T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)")
                    println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | s = $(s) | k = $(K)")
                    main(t_f, r_max, s, K)
                    cd("../../..")
                else
                    println("This is already calculated")
                end
            end
        end
    end
end

Calculating T_Final=150000 | R_Max = 3.5 | s = 1.9 | k = 0.005
  0.534251 seconds (791.03 k allocations: 46.136 MiB, 2.03% gc time, 94.58% compilation time)


┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007f8640f2c2b0.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArraysCore /home/nicomosty/.julia/packages/GPUArraysCore/lojQM/src/GPUArraysCore.jl:90


------------Init()-----------


Progress: 100%|█████████████████████████| Time: 0:14:34m12m


  0.110928 seconds (287.09 k allocations: 19.412 MiB, 93.85% compilation time)
